# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096955


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:50,  3.93s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:17,  2.87s/it]

Rendering models:  13%|█▎        | 4/30 [00:09<01:01,  2.38s/it]

Rendering models:  17%|█▋        | 5/30 [00:10<00:47,  1.88s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:31<00:02,  1.13it/s]

Rendering models:  93%|█████████▎| 28/30 [00:32<00:02,  1.05s/it]

Rendering models:  97%|█████████▋| 29/30 [00:33<00:01,  1.05s/it]

Rendering models: 100%|██████████| 30/30 [00:35<00:00,  1.16s/it]

not-logged-in-3b0b929f4475fd276561      0.000274
not-logged-in-3b0b929f4475fd276561      0.000171
not-logged-in-808d8d7eb7071532ff8d      0.000302
Jonas_Cross                             0.000244
Galactea                                0.041446
izzyathogwarts                          0.076675
not-logged-in-74261f68b83c7ae82ce3      0.000349
gracie_solveig                          0.000218
jaksonA                               492.805372
Quarkgc                                 0.000171
TRAPPIST-LIFE                           0.001468
not-logged-in-468edef11712cfc1e24c      0.000625
not-logged-in-5f30b2bb9b7f9d49da41      0.000307
not-logged-in-57d4f5f471f34dd12df9      0.000164
not-logged-in-5f30b2bb9b7f9d49da41     46.685813
gabrielweiss                            0.004937
clairedeu                               0.000164
pangeli5                                0.000177
rhondacard                              0.000740
not-logged-in-04ed9ae880f18762cd9e      0.068547
Lavadude            

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())